In [1]:
# Install required packages (uncomment and run if needed)
!sudo apt update
!sudo apt install postgresql postgresql-contrib
!pip install psycopg2-binary pandas


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,726 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:13 https://ppa.launchpadconten

In [ ]:
#use this in following box, to quit \q

CREATE USER de_user WITH PASSWORD 'de_pass';
CREATE DATABASE de_course OWNER de_user;
GRANT ALL PRIVILEGES ON DATABASE de_course TO de_user;

In [2]:
!sudo service postgresql start
!sudo -u postgres psql


 * Starting PostgreSQL 14 database server
   ...done.
psql (14.17 (Ubuntu 14.17-0ubuntu0.22.04.1))
Type "help" for help.

postgres=# CREATE USER de_user WITH PASSWORD 'de_pass'; CREATE DATABASE de_course OWNER de_user; GRANT ALL PRIVILEGES ON DATABASE de_course TO de_user;
CREATE ROLE
CREATE DATABASE
GRANT
postgres=# \q


In [3]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="de_course",
    user="de_user",
    password="de_pass"
)

cur = conn.cursor()
print("✅ Connected to PostgreSQL!")


✅ Connected to PostgreSQL!


In [18]:
# using faker library to create two csv files, one for orders one for users

!pip install Faker pandas
from faker import Faker
import pandas as pd

fake = Faker()

# Create Users CSV
num_users = 100
users_data = []
for _ in range(num_users):
    users_data.append({
        'user_id': fake.unique.random_int(min=1, max=100000),
        'username': fake.user_name(),
        'email': fake.email(),
        'registration_date': fake.date_time_this_year()
    })

users_df = pd.DataFrame(users_data)
users_df.to_csv('users.csv', index=False)
print("Created users.csv")

# Create Orders CSV
num_orders = 500
orders_data = []
for _ in range(num_orders):
    orders_data.append({
        'order_id': fake.unique.random_int(min=1, max=500000),
        'user_id': fake.random_element(elements=users_df['user_id'].tolist()), # Link to existing user_ids
        'order_date': fake.date_time_this_year(),
        'total_amount': round(fake.random_number(digits=4, fix_len=False) / 100.0, 2),
        'product_name': fake.word() + " " + fake.word()
    })

orders_df = pd.DataFrame(orders_data)
orders_df.to_csv('orders.csv', index=False)
print("Created orders.csv")

# Example: Display first few rows of the created files
print("\n--- users.csv ---")
print(users_df.head())

print("\n--- orders.csv ---")
print(orders_df.head())

Created users.csv
Created orders.csv

--- users.csv ---
   user_id         username                       email  \
0    85194    davisjennifer     wilcoxjenny@example.org   
1    43961        rebecca37  collinsmatthew@example.org   
2    84150         steven81      paulwilson@example.com   
3     7996  jasminecarrillo          colson@example.net   
4    89840     jeffreysmith       jeffrey93@example.net   

           registration_date  
0 2025-02-13 10:47:57.135925  
1 2025-02-20 13:14:20.053655  
2 2025-01-22 15:51:01.806412  
3 2025-02-14 22:09:48.378276  
4 2025-04-18 00:15:57.551806  

--- orders.csv ---
   order_id  user_id                 order_date  total_amount  \
0    174845    95692 2025-04-06 11:11:05.697651         58.99   
1    225459     6342 2025-04-02 15:54:18.416565         45.75   
2    170092    57248 2025-03-25 01:18:09.647095         71.64   
3    211303    20236 2025-05-18 12:52:09.813660         54.26   
4     61100    62017 2025-01-14 00:24:42.523763         18

In [42]:
# Drop tables if they exist
# If a previous command failed, the transaction might be aborted.
# Rollback the transaction to clear the error state.
conn.rollback()
cur.execute("DROP TABLE IF EXISTS orders;")
cur.execute("DROP TABLE IF EXISTS users;")

# Create tables
cur.execute("""
CREATE TABLE users (
    user_id INT PRIMARY KEY,
    username TEXT,
    email TEXT,
    registration_date DATE
);
""")

cur.execute("""
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    user_id INT REFERENCES users(user_id),
    total_amount NUMERIC,
    order_date DATE,
    product_name TEXT
);
""")

conn.commit()
print("✅ Tables created!")

✅ Tables created!


In [43]:
# Read from CSVs created using Faker
import pandas as pd

users_df = pd.read_csv("users.csv")  # Path to your users data
orders_df = pd.read_csv("orders.csv")  # Path to your orders data

# Insert into users
for _, row in users_df.iterrows():
    cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s)", tuple(row))

# Insert into orders
# Ensure the column order in the tuple matches the table definition:
# order_id INT, user_id INT, total_amount NUMERIC, order_date DATE, product_name TEXT
for _, row in orders_df.iterrows():
    cur.execute(
        "INSERT INTO orders (order_id, user_id, total_amount, order_date, product_name) VALUES (%s, %s, %s, %s, %s)",
        (row['order_id'], row['user_id'], row['total_amount'], row['order_date'], row['product_name'])
    )

conn.commit()
print("✅ Sample data inserted!")

✅ Sample data inserted!


In [44]:
# Basic SELECT
cur.execute("SELECT * FROM users;")
for row in cur.fetchall():
    print(row)


(85194, 'davisjennifer', 'wilcoxjenny@example.org', datetime.date(2025, 2, 13))
(43961, 'rebecca37', 'collinsmatthew@example.org', datetime.date(2025, 2, 20))
(84150, 'steven81', 'paulwilson@example.com', datetime.date(2025, 1, 22))
(7996, 'jasminecarrillo', 'colson@example.net', datetime.date(2025, 2, 14))
(89840, 'jeffreysmith', 'jeffrey93@example.net', datetime.date(2025, 4, 18))
(5421, 'mcintyremaria', 'richardmorris@example.net', datetime.date(2025, 4, 8))
(29843, 'kathy36', 'sjenkins@example.net', datetime.date(2025, 4, 12))
(95031, 'wharrison', 'holmesamanda@example.org', datetime.date(2025, 5, 12))
(75080, 'esmith', 'amandabailey@example.net', datetime.date(2025, 4, 8))
(62963, 'tlucas', 'maustin@example.org', datetime.date(2025, 3, 20))
(98399, 'amy46', 'aguilartimothy@example.net', datetime.date(2025, 3, 11))
(82363, 'harrellroger', 'carolsmith@example.com', datetime.date(2025, 3, 14))
(25853, 'wallacecorey', 'xlee@example.org', datetime.date(2025, 1, 9))
(33688, 'robersonpam

In [45]:
# Filtering
cur.execute("SELECT * FROM orders WHERE total_amount > 80;")
print(cur.fetchall())


[(169059, 21641, Decimal('92.17'), datetime.date(2025, 5, 2), 'collection modern'), (417253, 24320, Decimal('80.35'), datetime.date(2025, 1, 28), 'outside loss'), (253134, 77212, Decimal('99.2'), datetime.date(2025, 1, 15), 'attack again'), (313646, 21641, Decimal('98.6'), datetime.date(2025, 3, 7), 'occur pressure'), (203212, 2870, Decimal('94.05'), datetime.date(2025, 3, 16), 'office boy'), (107629, 84204, Decimal('97.53'), datetime.date(2025, 1, 21), 'history power'), (25978, 75080, Decimal('85.76'), datetime.date(2025, 3, 20), 'movie detail'), (191444, 20536, Decimal('91.34'), datetime.date(2025, 1, 30), 'possible total'), (16652, 6342, Decimal('99.64'), datetime.date(2025, 2, 20), 'increase century'), (243606, 84150, Decimal('86.42'), datetime.date(2025, 5, 1), 'case pretty'), (174782, 98062, Decimal('89.92'), datetime.date(2025, 3, 11), 'commercial morning'), (297900, 54570, Decimal('86.19'), datetime.date(2025, 1, 12), 'modern research'), (151046, 85537, Decimal('94.72'), dateti

In [46]:
# Sorting and limiting
cur.execute("SELECT * FROM orders ORDER BY total_amount DESC LIMIT 5;")
print(cur.fetchall())

[(248441, 6462, Decimal('99.9'), datetime.date(2025, 4, 15), 'record late'), (147811, 50576, Decimal('99.9'), datetime.date(2025, 4, 23), 'example nor'), (16652, 6342, Decimal('99.64'), datetime.date(2025, 2, 20), 'increase century'), (98795, 44195, Decimal('99.46'), datetime.date(2025, 1, 10), 'dog he'), (426555, 85537, Decimal('99.38'), datetime.date(2025, 4, 3), 'interview radio')]


In [49]:
# JOIN users and orders
conn.rollback() # لو علق
cur.execute("""
SELECT users.username, orders.total_amount, orders.order_date
FROM users
JOIN orders ON users.user_id = orders.user_id;
""")
print(cur.fetchall())

[('sergio82', Decimal('58.99'), datetime.date(2025, 4, 6)), ('smcpherson', Decimal('45.75'), datetime.date(2025, 4, 2)), ('adamsbelinda', Decimal('71.64'), datetime.date(2025, 3, 25)), ('laura04', Decimal('54.26'), datetime.date(2025, 5, 18)), ('brownmadison', Decimal('18.69'), datetime.date(2025, 1, 14)), ('nataliebuchanan', Decimal('75.41'), datetime.date(2025, 3, 19)), ('abray', Decimal('92.17'), datetime.date(2025, 5, 2)), ('rbrown', Decimal('1.33'), datetime.date(2025, 4, 13)), ('gardnercharles', Decimal('41.36'), datetime.date(2025, 1, 17)), ('fdominguez', Decimal('43.06'), datetime.date(2025, 5, 6)), ('gordonsarah', Decimal('71.81'), datetime.date(2025, 1, 13)), ('ahurst', Decimal('80.35'), datetime.date(2025, 1, 28)), ('rogershenry', Decimal('72.76'), datetime.date(2025, 4, 18)), ('mcintyremaria', Decimal('15.7'), datetime.date(2025, 2, 10)), ('travisbarajas', Decimal('52.39'), datetime.date(2025, 1, 14)), ('barrydean', Decimal('2.74'), datetime.date(2025, 3, 29)), ('gardnercha

In [52]:
# GROUP BY and aggregation
conn.rollback()
cur.execute("""
SELECT user_id, COUNT(*) AS total_orders, SUM(total_amount) AS total_spent -- Changed 'amount' to 'total_amount'
FROM orders
GROUP BY user_id;
""")
print(cur.fetchall())

[(65575, 3, Decimal('192.07')), (37639, 3, Decimal('115.40')), (98062, 4, Decimal('361.68')), (21641, 6, Decimal('382.05')), (44195, 4, Decimal('270.88')), (88403, 6, Decimal('383.40')), (73311, 5, Decimal('180.37')), (51373, 5, Decimal('149.49')), (57248, 6, Decimal('322.33')), (82363, 4, Decimal('176.98')), (64737, 8, Decimal('335.16')), (15602, 2, Decimal('83.35')), (63683, 2, Decimal('130.09')), (25853, 5, Decimal('223.37')), (7715, 7, Decimal('411.22')), (25018, 2, Decimal('137.99')), (20236, 4, Decimal('225.68')), (88065, 9, Decimal('418.50')), (50968, 5, Decimal('213.57')), (6342, 7, Decimal('400.75')), (71181, 4, Decimal('184.50')), (47004, 3, Decimal('117.56')), (55777, 5, Decimal('384.23')), (62017, 5, Decimal('141.80')), (2870, 4, Decimal('231.19')), (81769, 2, Decimal('141.52')), (55548, 2, Decimal('23.97')), (32455, 7, Decimal('386.26')), (35404, 5, Decimal('297.90')), (50576, 5, Decimal('283.46')), (15428, 6, Decimal('316.99')), (11496, 4, Decimal('106.96')), (54570, 5, D

In [53]:
cur.close()
conn.close()
print("✅ PostgreSQL connection closed.")

✅ PostgreSQL connection closed.
